<div align="center">

```
██╗   ██╗███████╗██████╗ ██╗████████╗ █████╗ ███████╗
██║   ██║██╔════╝██╔══██╗██║╚══██╔══╝██╔══██╗██╔════╝
██║   ██║█████╗  ██████╔╝██║   ██║   ███████║███████╗
╚██╗ ██╔╝██╔══╝  ██╔══██╗██║   ██║   ██╔══██║╚════██║
 ╚████╔╝ ███████╗██║  ██║██║   ██║   ██║  ██║███████║
  ╚═══╝  ╚══════╝╚═╝  ╚═╝╚═╝   ╚═╝   ╚═╝  ╚═╝╚══════╝
```

# 🧠 AI Lie Detection System — Multimodal Behavioral Analysis
### *Computer Vision · Voice Stress · Micro-Expressions · NLP Fusion*

---

| Channel | Technology | Weight |
|---|---|:---:|
| 😶 Micro-Expressions | OpenCV + Rule Engine | 25% |
| 👁 Eye Movement & Blink Rate | OpenCV Landmark Tracking | 20% |
| 🎙 Voice Stress Patterns | Signal Processing (FFT) | 25% |
| 🧠 NLP Sentiment + Deception | Linguistic Pattern Engine | 15% |
| ⏱ Response Delay / Cognitive Load | Timing Analysis | 10% |
| 💓 Physiological Estimation | CV-derived signals | 5% |

> **Truth Score = Weighted Fusion of all 6 channels → 0–100% confidence**

</div>

---
## 📋 Table of Contents

1. [🔧 Environment Setup & Imports](#1-environment-setup)
2. [⚙️ Configuration & Constants](#2-configuration)
3. [🛠 Utility Functions](#3-utilities)
4. [😶 Module 1 — Micro-Expression Detector](#4-micro-expression)
5. [👁 Module 2 — Eye Movement & Blink Analyzer](#5-eye-movement)
6. [🎙 Module 3 — Voice Stress Analyzer](#6-voice-stress)
7. [🧠 Module 4 — NLP Deception Engine](#7-nlp-engine)
8. [⏱ Module 5 — Response Delay Tracker](#8-response-delay)
9. [💓 Module 6 — Physiological Signal Estimator](#9-physiological)
10. [🔀 Score Fusion Engine](#10-score-fusion)
11. [📊 Visualization Dashboard](#11-visualization)
12. [🎬 Full Pipeline — Run Analysis](#12-full-pipeline)
13. [📄 Report Generator & Export](#13-report)
14. [🧪 Demo — Synthetic Simulation](#14-demo)

---

---
<a id="1-environment-setup"></a>
## 🔧 Section 1 — Environment Setup & Imports

Install all required packages and import dependencies.

In [ ]:
# ============================================================
# VERITAS — AI Deception Analysis System
# Install dependencies (run once)
# ============================================================

import subprocess, sys

def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])

required = [
    "opencv-python-headless",
    "numpy",
    "pandas", 
    "matplotlib",
    "seaborn",
    "scipy",
    "scikit-learn",
    "Pillow",
    "tqdm",
    "ipywidgets",
]

print("📦 Installing dependencies...")
for pkg in required:
    try:
        install(pkg)
        print(f"  ✅ {pkg}")
    except Exception as e:
        print(f"  ⚠️  {pkg} — {e}")

print("\n🚀 All packages ready!")

In [ ]:
# ============================================================
# Core Imports
# ============================================================

# Standard Library
import os
import sys
import time
import json
import math
import random
import hashlib
import warnings
import datetime
from pathlib import Path
from collections import deque, Counter
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Tuple, Optional, Union
warnings.filterwarnings("ignore")

# Numeric & Scientific
import numpy as np
import pandas as pd
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.stats import zscore, pearsonr
from scipy.signal import butter, filtfilt, find_peaks

# Computer Vision
import cv2

# Machine Learning
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression

# Visualization
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.patches import FancyBboxPatch, Circle, Arc
from matplotlib.animation import FuncAnimation
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

# Image Processing
from PIL import Image, ImageDraw, ImageFont

# Progress & Display
from tqdm.notebook import tqdm
from IPython.display import display, HTML, clear_output, Image as IPImage
import ipywidgets as widgets

# ─── Style Configuration ───────────────────────────────────────
plt.style.use("dark_background")
matplotlib.rcParams.update({
    "figure.facecolor":  "#030812",
    "axes.facecolor":    "#080f1e",
    "axes.edgecolor":    "#0d2044",
    "axes.labelcolor":   "#a8c8e8",
    "text.color":        "#a8c8e8",
    "xtick.color":       "#3a5a7a",
    "ytick.color":       "#3a5a7a",
    "grid.color":        "#0d2044",
    "grid.linewidth":    0.5,
    "font.family":       "monospace",
    "axes.titlesize":    13,
    "axes.titleweight":  "bold",
    "figure.dpi":        120,
})

CYAN   = "#00d4ff"
RED    = "#ff3c6e"
GREEN  = "#39ff14"
YELLOW = "#ffb800"
ORANGE = "#ff6b35"
DIM    = "#3a5a7a"

print("✅ All imports successful!")
print(f"   NumPy  {np.__version__}  |  OpenCV {cv2.__version__}  |  Pandas {pd.__version__}  |  Matplotlib {matplotlib.__version__}")

---
<a id="2-configuration"></a>
## ⚙️ Section 2 — Configuration & Constants

In [ ]:
# ============================================================
# VERITAS SYSTEM CONFIGURATION
# ============================================================

@dataclass
class VeritasConfig:
    """Central configuration for the VERITAS system."""

    # ─── Channel Weights (must sum to 1.0) ───────────────────
    WEIGHT_MICRO_EXPR:  float = 0.25
    WEIGHT_EYE_MOVE:    float = 0.20
    WEIGHT_VOICE_STRESS:float = 0.25
    WEIGHT_NLP:         float = 0.15
    WEIGHT_DELAY:       float = 0.10
    WEIGHT_PHYSIO:      float = 0.05

    # ─── Verdict Thresholds ──────────────────────────────────
    TRUTHFUL_THRESHOLD:        float = 75.0
    LIKELY_TRUTHFUL_THRESHOLD: float = 55.0
    INCONCLUSIVE_THRESHOLD:    float = 40.0
    LIKELY_DECEPTIVE_THRESHOLD:float = 25.0
    # Below 25 = DECEPTIVE

    # ─── Eye Tracking ────────────────────────────────────────
    NORMAL_BLINK_MIN: int = 10   # blinks/min
    NORMAL_BLINK_MAX: int = 20   # blinks/min
    ELEVATED_BLINK:   int = 30   # blinks/min — anxiety threshold
    GAZE_STABLE_THRESHOLD: float = 0.70  # fraction of time looking forward

    # ─── Voice Stress ─────────────────────────────────────────
    SAMPLE_RATE: int = 22050
    FFT_WINDOW:  int = 2048
    STRESS_FREQ_LOW:  float = 300.0   # Hz — voice fundamental
    STRESS_FREQ_HIGH: float = 3000.0  # Hz — voice harmonic range
    TREMOR_THRESHOLD: float = 0.15    # normalized tremor index

    # ─── Response Delay ─────────────────────────────────────
    FAST_RESPONSE:    float = 0.4   # seconds — spontaneous
    NORMAL_RESPONSE:  float = 0.7   # seconds — normal processing
    SLOW_RESPONSE:    float = 1.0   # seconds — elevated cognitive load
    HIGH_LATENCY:     float = 1.5   # seconds — strong deception marker

    # ─── Smoothing ───────────────────────────────────────────
    EMA_ALPHA: float = 0.12   # exponential moving average factor

    # ─── Session ─────────────────────────────────────────────
    TIMELINE_BUFFER: int = 60   # data points in rolling timeline
    FEED_MAX_EVENTS: int = 200  # max behavioral feed entries

    # ─── NLP ─────────────────────────────────────────────────
    HEDGE_RISK_WEIGHT:   float = 4.0
    PROTEST_RISK_WEIGHT: float = 8.0
    FILLER_RISK_WEIGHT:  float = 1.5
    NEGATION_RISK_WEIGHT:float = 3.0
    DATE_TRUTH_WEIGHT:   float = 3.0
    NUMBER_TRUTH_WEIGHT: float = 2.0

    def validate(self):
        total = (self.WEIGHT_MICRO_EXPR + self.WEIGHT_EYE_MOVE +
                 self.WEIGHT_VOICE_STRESS + self.WEIGHT_NLP +
                 self.WEIGHT_DELAY + self.WEIGHT_PHYSIO)
        assert abs(total - 1.0) < 1e-9, f"Weights must sum to 1.0, got {total:.4f}"
        print(f"  ✅ Config validated — weights sum: {total:.2f}")


CFG = VeritasConfig()
CFG.validate()
print(f"  ✅ VERITAS Config loaded")
print(f"\n  Channel Weights:")
print(f"    Micro-Expression : {CFG.WEIGHT_MICRO_EXPR:.0%}")
print(f"    Eye Movement     : {CFG.WEIGHT_EYE_MOVE:.0%}")
print(f"    Voice Stress     : {CFG.WEIGHT_VOICE_STRESS:.0%}")
print(f"    NLP Analysis     : {CFG.WEIGHT_NLP:.0%}")
print(f"    Response Delay   : {CFG.WEIGHT_DELAY:.0%}")
print(f"    Physiological    : {CFG.WEIGHT_PHYSIO:.0%}")

---
<a id="3-utilities"></a>
## 🛠 Section 3 — Utility Functions & Data Structures

In [ ]:
# ============================================================
# CORE DATA STRUCTURES
# ============================================================

@dataclass
class ChannelScore:
    """Score from a single analysis channel."""
    name: str
    truth_score: float          # 0–100, higher = more truthful
    deception_risk: float       # 0–100, higher = more deceptive
    confidence: float           # 0–1, how confident the engine is
    flags: List[str] = field(default_factory=list)
    metadata: Dict  = field(default_factory=dict)
    timestamp: float = field(default_factory=time.time)

    @property
    def label(self) -> str:
        if self.truth_score >= 75: return "TRUTHFUL"
        if self.truth_score >= 55: return "LIKELY TRUTHFUL"
        if self.truth_score >= 40: return "INCONCLUSIVE"
        if self.truth_score >= 25: return "LIKELY DECEPTIVE"
        return "DECEPTIVE"

    @property
    def color(self) -> str:
        if self.truth_score >= 75: return GREEN
        if self.truth_score >= 55: return CYAN
        if self.truth_score >= 40: return YELLOW
        if self.truth_score >= 25: return ORANGE
        return RED


@dataclass
class AnalysisResult:
    """Complete analysis result from the VERITAS engine."""
    session_id: str
    timestamp: datetime.datetime
    duration_seconds: float
    channels: Dict[str, ChannelScore]
    truth_score: float          # Weighted fusion score
    deception_risk: float
    verdict: str
    flags: List[str]
    timeline: List[float]       # Rolling truth score history
    raw_text: str = ""

    @property
    def verdict_color(self) -> str:
        if self.truth_score >= 75: return GREEN
        if self.truth_score >= 55: return CYAN
        if self.truth_score >= 40: return YELLOW
        if self.truth_score >= 25: return ORANGE
        return RED


# ─── Utility Functions ────────────────────────────────────────

class EMA:
    """Exponential Moving Average smoother."""
    def __init__(self, alpha: float = 0.12):
        self.alpha = alpha
        self.value = None

    def update(self, x: float) -> float:
        if self.value is None:
            self.value = x
        else:
            self.value = self.alpha * x + (1 - self.alpha) * self.value
        return self.value

    def reset(self):
        self.value = None


def clamp(x: float, lo: float = 0.0, hi: float = 100.0) -> float:
    return max(lo, min(hi, x))


def normalize(arr: np.ndarray, lo: float = 0, hi: float = 100) -> np.ndarray:
    mn, mx = arr.min(), arr.max()
    if mx == mn: return np.full_like(arr, (lo + hi) / 2, dtype=float)
    return lo + (arr - mn) / (mx - mn) * (hi - lo)


def verdict_from_score(score: float) -> str:
    if score >= CFG.TRUTHFUL_THRESHOLD:        return "✅ TRUTHFUL"
    if score >= CFG.LIKELY_TRUTHFUL_THRESHOLD: return "🔵 LIKELY TRUTHFUL"
    if score >= CFG.INCONCLUSIVE_THRESHOLD:    return "🟡 INCONCLUSIVE"
    if score >= CFG.LIKELY_DECEPTIVE_THRESHOLD:return "🟠 LIKELY DECEPTIVE"
    return "🔴 DECEPTIVE"


def color_from_score(score: float) -> str:
    if score >= 75: return GREEN
    if score >= 55: return CYAN
    if score >= 40: return YELLOW
    if score >= 25: return ORANGE
    return RED


def generate_session_id() -> str:
    return hashlib.md5(str(time.time()).encode()).hexdigest()[:8].upper()


print("✅ Data structures & utilities initialized")

---
<a id="4-micro-expression"></a>
## 😶 Module 1 — Micro-Expression Detector

Detects involuntary facial muscle movements lasting 40–500ms that leak suppressed emotions. Uses OpenCV face detection + optical flow analysis.

In [ ]:
# ============================================================
# MICRO-EXPRESSION DETECTION ENGINE
# ============================================================

class MicroExpressionDetector:
    """
    Detects micro-expressions using:
    - OpenCV Haar Cascade face detection
    - Lucas-Kanade Optical Flow for facial motion vectors
    - Expression classification via Action Unit (AU) heuristics
    - Duration gating (40–500ms = micro, longer = macro)
    """

    EXPRESSIONS = {
        "NEUTRAL":           {"risk": 0,  "description": "Baseline resting state"},
        "MICRO-FEAR":        {"risk": 85, "description": "Involuntary fear flash — strong deception signal"},
        "SUPPRESSED SMILE":  {"risk": 60, "description": "Duping delight — joy at deceiving"},
        "BROW FURROW":       {"risk": 45, "description": "Cognitive effort / discomfort"},
        "NOSTRIL FLARE":     {"risk": 70, "description": "Fight-or-flight activation"},
        "LIP COMPRESSION":   {"risk": 55, "description": "Suppressed speech / withheld info"},
        "EYE WIDENING":      {"risk": 20, "description": "Surprise — genuine emotional response"},
        "CONTEMPT":          {"risk": 75, "description": "One-sided smirk — disrespect or deception"},
        "DISGUST FLASH":     {"risk": 50, "description": "Brief disgust micro-expression"},
    }

    def __init__(self):
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
        )
        self.eye_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + "haarcascade_eye.xml"
        )
        self.prev_gray = None
        self.prev_points = None
        self.expression_history = deque(maxlen=50)
        self.motion_history = deque(maxlen=30)
        self.ema = EMA(alpha=0.15)
        self.frame_count = 0

    def detect_face(self, frame: np.ndarray) -> Optional[Tuple]:
        """Returns (x, y, w, h) of largest detected face or None."""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5, minSize=(80, 80)
        )
        if len(faces) == 0:
            return None, gray
        # Return largest face
        face = max(faces, key=lambda f: f[2] * f[3])
        return face, gray

    def compute_optical_flow(self, gray: np.ndarray) -> float:
        """Compute mean optical flow magnitude — proxy for facial motion."""
        if self.prev_gray is None or self.prev_points is None:
            self.prev_gray = gray
            # Select good feature points (corners)
            self.prev_points = cv2.goodFeaturesToTrack(
                gray, maxCorners=100, qualityLevel=0.01, minDistance=10
            )
            return 0.0

        if self.prev_points is None or len(self.prev_points) < 5:
            self.prev_gray = gray
            return 0.0

        new_points, status, _ = cv2.calcOpticalFlowPyrLK(
            self.prev_gray, gray, self.prev_points, None
        )
        if new_points is None:
            return 0.0

        good_old = self.prev_points[status == 1]
        good_new = new_points[status == 1]

        if len(good_new) < 3:
            return 0.0

        displacements = np.linalg.norm(good_new - good_old, axis=1)
        mean_motion = float(np.mean(displacements))

        self.prev_gray = gray
        self.prev_points = cv2.goodFeaturesToTrack(
            gray, maxCorners=100, qualityLevel=0.01, minDistance=10
        )
        return mean_motion

    def classify_expression(self, motion: float, face: Optional[Tuple]) -> str:
        """
        Classify expression from motion magnitude + face region analysis.
        In production: replace with CNN or FaceMesh AU detector.
        """
        self.motion_history.append(motion)

        if len(self.motion_history) < 5:
            return "NEUTRAL"

        mean_m = np.mean(self.motion_history)
        std_m  = np.std(self.motion_history)

        # Spike detection (micro-expression = sudden deviation from baseline)
        is_spike = (motion > mean_m + 2.0 * std_m) and (motion > 1.5)

        if not is_spike:
            return "NEUTRAL"

        # Classify spike into expression type (heuristic)
        r = random.random()
        expressions = list(self.EXPRESSIONS.keys())
        weights     = [1 if e == "NEUTRAL" else 2 for e in expressions]
        # Higher motion = stronger expression
        if motion > 4.0:
            high_risk = ["MICRO-FEAR", "NOSTRIL FLARE", "CONTEMPT"]
            return random.choice(high_risk)
        elif motion > 2.5:
            mid_risk = ["SUPPRESSED SMILE", "LIP COMPRESSION", "DISGUST FLASH", "BROW FURROW"]
            return random.choice(mid_risk)
        else:
            return random.choice(["BROW FURROW", "EYE WIDENING", "LIP COMPRESSION"])

    def analyze_frame(self, frame: np.ndarray) -> ChannelScore:
        """Process one video frame and return micro-expression score."""
        self.frame_count += 1
        face, gray = self.detect_face(frame)
        motion = self.compute_optical_flow(gray)
        expression = self.classify_expression(motion, face)
        self.expression_history.append(expression)

        # Compute truth score from expression risk
        risk = self.EXPRESSIONS[expression]["risk"]
        recent_risks = [self.EXPRESSIONS[e]["risk"]
                        for e in list(self.expression_history)[-10:]]
        avg_risk = np.mean(recent_risks) if recent_risks else 0
        truth = clamp(100 - avg_risk - random.uniform(0, 10))
        truth = self.ema.update(truth)

        flags = []
        if avg_risk > 50:
            flags.append(f"HIGH-RISK EXPRESSION: {expression}")
        if avg_risk > 30:
            flags.append("ELEVATED AROUSAL DETECTED")

        return ChannelScore(
            name="Micro-Expression",
            truth_score=truth,
            deception_risk=clamp(100 - truth),
            confidence=0.85 if face is not None else 0.40,
            flags=flags,
            metadata={
                "expression": expression,
                "motion_magnitude": round(motion, 3),
                "description": self.EXPRESSIONS[expression]["description"],
                "face_detected": face is not None,
                "frames_analyzed": self.frame_count,
            }
        )

    def analyze_image(self, img_path: str) -> ChannelScore:
        """Analyze a single image file."""
        frame = cv2.imread(img_path)
        if frame is None:
            raise FileNotFoundError(f"Cannot load image: {img_path}")
        return self.analyze_frame(frame)

    def analyze_synthetic(self, n_frames: int = 100,
                           stress_level: float = 0.5) -> List[ChannelScore]:
        """
        Generate synthetic analysis results for demo/testing.
        stress_level: 0.0 (calm/truthful) → 1.0 (stressed/deceptive)
        """
        scores = []
        frame_h, frame_w = 480, 640
        
        for i in range(n_frames):
            frame = np.zeros((frame_h, frame_w, 3), dtype=np.uint8)
            # Add synthetic noise to simulate facial motion
            noise = np.random.normal(0, 20 + stress_level * 40, frame.shape).astype(np.uint8)
            frame = cv2.add(frame, noise)
            # Draw a synthetic "face" ellipse
            cx, cy = frame_w//2 + int(np.sin(i*0.1) * 5), frame_h//2
            cv2.ellipse(frame, (cx, cy), (80, 100), 0, 0, 360, (200, 180, 160), -1)
            scores.append(self.analyze_frame(frame))
        
        return scores


# ─── Instantiate & Quick Test ─────────────────────────────────
micro_detector = MicroExpressionDetector()

# Test with a synthetic frame
test_frame = np.random.randint(50, 200, (480, 640, 3), dtype=np.uint8)
test_score = micro_detector.analyze_frame(test_frame)

print("✅ MicroExpressionDetector ready")
print(f"   Expression Catalog  : {len(MicroExpressionDetector.EXPRESSIONS)} types")
print(f"   Test frame result   : {test_score.expression if hasattr(test_score, 'expression') else test_score.metadata.get('expression', 'N/A')}")
print(f"   Test truth score    : {test_score.truth_score:.1f}%")

---
<a id="5-eye-movement"></a>
## 👁 Module 2 — Eye Movement & Blink Analyzer

Tracks blink rate, gaze direction, and fixation stability — strong involuntary signals of arousal and cognitive deception.

In [ ]:
# ============================================================
# EYE MOVEMENT & BLINK ANALYSIS ENGINE
# ============================================================

class EyeMovementAnalyzer:
    """
    Analyzes:
    - Blink rate (elevated = arousal/stress)
    - Eye Aspect Ratio (EAR) for blink detection
    - Gaze direction (leftward/rightward shift = confabulation)
    - Gaze stability (unstable = evasive behavior)
    - Pupil dilation estimation
    """

    EAR_BLINK_THRESHOLD = 0.20   # below this = eye closed
    EAR_OPEN_MIN        = 0.25

    def __init__(self):
        self.eye_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + "haarcascade_eye.xml"
        )
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
        )
        self.blink_count      = 0
        self.session_start    = time.time()
        self.gaze_history     = deque(maxlen=100)   # x offset history
        self.blink_times      = deque(maxlen=50)
        self.was_blinking     = False
        self.ear_history      = deque(maxlen=30)
        self.ema              = EMA(alpha=0.10)
        self.frame_times      = deque(maxlen=200)

    def compute_ear(self, eye_points: np.ndarray) -> float:
        """Eye Aspect Ratio (EAR) from 6 landmark points."""
        # EAR = (||p2-p6|| + ||p3-p5||) / (2 * ||p1-p4||)
        if len(eye_points) < 6:
            return 0.3  # default open
        A = np.linalg.norm(eye_points[1] - eye_points[5])
        B = np.linalg.norm(eye_points[2] - eye_points[4])
        C = np.linalg.norm(eye_points[0] - eye_points[3])
        return (A + B) / (2.0 * C) if C > 0 else 0.3

    def detect_blink(self, ear: float) -> bool:
        """Detect blink via EAR threshold crossing."""
        is_blink = ear < self.EAR_BLINK_THRESHOLD
        if is_blink and not self.was_blinking:
            self.blink_count += 1
            self.blink_times.append(time.time())
        self.was_blinking = is_blink
        return is_blink

    def blink_rate_bpm(self) -> float:
        """Blinks per minute over session so far."""
        elapsed = max(1.0, time.time() - self.session_start)
        return (self.blink_count / elapsed) * 60

    def gaze_direction(self, eyes: list, frame_w: int) -> str:
        """Classify gaze direction from eye center positions."""
        if not eyes:
            return "UNKNOWN"
        centers = [(ex + ew//2) for (ex, ey, ew, eh) in eyes]
        avg_x = np.mean(centers)
        center = frame_w / 2
        offset = (avg_x - center) / center  # -1 (far left) to +1 (far right)
        self.gaze_history.append(offset)

        if offset < -0.15:
            return "LEFT"     # leftward — memory retrieval (truth signal)
        elif offset > 0.15:
            return "RIGHT"    # rightward — construction (deception signal)
        return "CENTER"

    def gaze_stability(self) -> float:
        """Fraction of frames where gaze was centered."""
        if not self.gaze_history:
            return 1.0
        stable = sum(1 for g in self.gaze_history if abs(g) < 0.15)
        return stable / len(self.gaze_history)

    def analyze_frame(self, frame: np.ndarray) -> ChannelScore:
        """Analyze one frame for eye movement signals."""
        self.frame_times.append(time.time())
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        h, w = frame.shape[:2]

        # Face detection
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(60, 60))
        eyes = []
        if len(faces) > 0:
            fx, fy, fw, fh = max(faces, key=lambda f: f[2]*f[3])
            roi = gray[fy:fy+fh, fx:fx+fw]
            detected_eyes = self.eye_cascade.detectMultiScale(roi, 1.1, 3)
            eyes = [(fx+ex, fy+ey, ew, eh) for (ex, ey, ew, eh) in detected_eyes]

        # Simulate EAR (in production: use FaceMesh or dlib 68-point)
        simulated_ear = 0.30 + random.gauss(0, 0.05)
        simulated_ear = max(0.05, min(0.45, simulated_ear))
        self.ear_history.append(simulated_ear)
        self.detect_blink(simulated_ear)

        gaze = self.gaze_direction(eyes, w)
        stability = self.gaze_stability()
        bpm = self.blink_rate_bpm()

        # ─── Score Computation ─────────────────────────────
        # Blink rate component
        if bpm < CFG.NORMAL_BLINK_MAX:
            blink_trust = 100
        elif bpm < CFG.ELEVATED_BLINK:
            blink_trust = 100 - (bpm - CFG.NORMAL_BLINK_MAX) * 3
        else:
            blink_trust = max(0, 100 - (bpm - CFG.ELEVATED_BLINK) * 5 - 20)

        # Gaze component
        gaze_trust = stability * 100

        # Rightward gaze penalty (confabulation indicator)
        if gaze == "RIGHT":
            gaze_trust -= 20
        elif gaze == "LEFT":
            gaze_trust += 5  # memory retrieval = truthful signal

        truth = clamp(blink_trust * 0.55 + gaze_trust * 0.45)
        truth = self.ema.update(truth)

        flags = []
        if bpm > CFG.ELEVATED_BLINK:
            flags.append(f"ELEVATED BLINK RATE: {bpm:.0f} bpm (threshold: {CFG.ELEVATED_BLINK})")
        if stability < 0.60:
            flags.append("EVASIVE GAZE PATTERN DETECTED")
        if gaze == "RIGHT":
            flags.append("RIGHTWARD GAZE — POTENTIAL CONFABULATION")

        return ChannelScore(
            name="Eye Movement",
            truth_score=truth,
            deception_risk=clamp(100 - truth),
            confidence=0.80 if len(eyes) >= 2 else 0.50,
            flags=flags,
            metadata={
                "blink_count": self.blink_count,
                "blink_rate_bpm": round(bpm, 1),
                "gaze_direction": gaze,
                "gaze_stability": round(stability, 3),
                "eyes_detected": len(eyes),
                "ear": round(simulated_ear, 3),
                "normal_range": f"{CFG.NORMAL_BLINK_MIN}–{CFG.NORMAL_BLINK_MAX} bpm",
            }
        )

    def analyze_synthetic(self, n_frames: int = 100,
                           stress_level: float = 0.5) -> List[ChannelScore]:
        """Synthetic demo analysis."""
        scores = []
        for _ in range(n_frames):
            frame = np.random.randint(0, 200, (480, 640, 3), dtype=np.uint8)
            # Stress increases blink frequency simulation
            if random.random() < stress_level * 0.3:
                self.blink_count += 1
            scores.append(self.analyze_frame(frame))
        return scores


eye_analyzer = EyeMovementAnalyzer()
test_eye = eye_analyzer.analyze_frame(test_frame)

print("✅ EyeMovementAnalyzer ready")
print(f"   Blink Rate Normals  : {CFG.NORMAL_BLINK_MIN}–{CFG.NORMAL_BLINK_MAX} bpm")
print(f"   Elevated Threshold  : >{CFG.ELEVATED_BLINK} bpm")
print(f"   Test result — Gaze  : {test_eye.metadata['gaze_direction']}")
print(f"   Test truth score    : {test_eye.truth_score:.1f}%")

---
<a id="6-voice-stress"></a>
## 🎙 Module 3 — Voice Stress Analyzer

Uses Fast Fourier Transform (FFT) to analyze audio frequency content. Elevated high-frequency energy, pitch variance, and tremor patterns are strong stress markers.

In [ ]:
# ============================================================
# VOICE STRESS ANALYSIS ENGINE
# ============================================================

class VoiceStressAnalyzer:
    """
    Analyzes audio signals for:
    - Fundamental frequency (F0) variance — pitch instability
    - High-frequency tremor index (8–14 Hz modulation)
    - Harmonics-to-Noise Ratio (HNR) estimation
    - Jitter (cycle-to-cycle frequency variation)
    - Shimmer (cycle-to-cycle amplitude variation)
    - Speech energy distribution across frequency bands
    """

    STRESS_BANDS = {
        "sub_bass":   (20,   300),
        "voice_fund": (300,  800),
        "voice_harm": (800,  2500),
        "sibilance":  (2500, 5000),
        "air":        (5000, 8000),
    }

    def __init__(self, sample_rate: int = 22050):
        self.sr = sample_rate
        self.f0_history         = deque(maxlen=100)
        self.energy_history     = deque(maxlen=100)
        self.stress_history     = deque(maxlen=100)
        self.ema                = EMA(alpha=0.15)
        self.segment_count      = 0

    def estimate_f0(self, audio: np.ndarray) -> float:
        """
        Estimate fundamental frequency (F0) using autocorrelation.
        Returns frequency in Hz, 0 if no voice detected.
        """
        # High-pass filter to remove DC
        audio_hp = audio - np.mean(audio)
        if np.max(np.abs(audio_hp)) < 0.01:
            return 0.0  # silence

        # Autocorrelation
        corr = np.correlate(audio_hp, audio_hp, mode='full')
        corr = corr[len(corr)//2:]

        # Find first peak after minimum lag (equiv. to max 500Hz)
        min_lag = int(self.sr / 500)  # 500 Hz max
        max_lag = int(self.sr / 60)   # 60 Hz min

        if max_lag > len(corr):
            max_lag = len(corr) - 1

        peak_region = corr[min_lag:max_lag]
        if len(peak_region) == 0:
            return 0.0

        peaks, _ = find_peaks(peak_region, height=np.max(peak_region) * 0.4)
        if len(peaks) == 0:
            return 0.0

        lag = peaks[0] + min_lag
        f0 = self.sr / lag
        return float(f0)

    def compute_fft_bands(self, audio: np.ndarray) -> Dict[str, float]:
        """Compute energy in each frequency band via FFT."""
        n = len(audio)
        yf = np.abs(fft(audio))[:n//2]
        xf = fftfreq(n, 1/self.sr)[:n//2]

        band_energies = {}
        for name, (lo, hi) in self.STRESS_BANDS.items():
            mask = (xf >= lo) & (xf < hi)
            band_energies[name] = float(np.mean(yf[mask])) if mask.any() else 0.0
        return band_energies

    def compute_tremor_index(self, audio: np.ndarray) -> float:
        """
        Detect 8–14 Hz voice tremor (pathological stress marker).
        High tremor index correlates with anxiety and deception stress.
        """
        # Envelope extraction via Hilbert transform
        analytic  = signal.hilbert(audio)
        envelope  = np.abs(analytic)

        # FFT of envelope — look for 8–14 Hz modulation
        env_fft   = np.abs(fft(envelope))[:len(envelope)//2]
        env_freqs = fftfreq(len(envelope), 1/self.sr)[:len(envelope)//2]

        tremor_mask  = (env_freqs >= 8) & (env_freqs <= 14)
        total_mask   = env_freqs > 0
        tremor_energy = env_fft[tremor_mask].sum() if tremor_mask.any() else 0
        total_energy  = env_fft[total_mask].sum()  if total_mask.any()  else 1
        return float(tremor_energy / total_energy) if total_energy > 0 else 0.0

    def compute_jitter(self, audio: np.ndarray) -> float:
        """
        Simplified jitter estimation — variance in zero-crossing rate.
        High jitter = pitch instability = stress marker.
        """
        zero_crossings = np.where(np.diff(np.sign(audio)))[0]
        if len(zero_crossings) < 2:
            return 0.0
        intervals = np.diff(zero_crossings)
        if np.mean(intervals) == 0:
            return 0.0
        return float(np.std(intervals) / np.mean(intervals))  # coefficient of variation

    def compute_shimmer(self, audio: np.ndarray) -> float:
        """
        Simplified shimmer — amplitude variation between peaks.
        High shimmer = vocal irregularity = stress.
        """
        peaks, _ = find_peaks(np.abs(audio), height=np.max(np.abs(audio)) * 0.3)
        if len(peaks) < 2:
            return 0.0
        amps = np.abs(audio)[peaks]
        return float(np.std(amps) / (np.mean(amps) + 1e-9))

    def analyze_segment(self, audio: np.ndarray) -> ChannelScore:
        """Analyze one audio segment (e.g. 0.5–2 seconds)."""
        self.segment_count += 1

        # Core measurements
        f0           = self.estimate_f0(audio)
        bands        = self.compute_fft_bands(audio)
        tremor_idx   = self.compute_tremor_index(audio)
        jitter       = self.compute_jitter(audio)
        shimmer      = self.compute_shimmer(audio)
        rms_energy   = float(np.sqrt(np.mean(audio**2)))

        self.f0_history.append(f0)
        self.stress_history.append(tremor_idx)

        # F0 variance component (higher variance = higher stress)
        f0_vals = [v for v in self.f0_history if v > 0]
        f0_var  = np.std(f0_vals) / (np.mean(f0_vals) + 1e-9) if f0_vals else 0
        f0_trust = clamp(100 - f0_var * 500)

        # Tremor component
        tremor_trust = clamp(100 - tremor_idx * 400)

        # Jitter component
        jitter_trust = clamp(100 - jitter * 60)

        # High frequency stress ratio
        voice_energy = bands["voice_fund"] + bands["voice_harm"]
        stress_ratio = bands["sibilance"] / (voice_energy + 1e-9)
        stress_trust = clamp(100 - stress_ratio * 50)

        # Weighted fusion
        truth = (f0_trust * 0.30 + tremor_trust * 0.35 +
                 jitter_trust * 0.20 + stress_trust * 0.15)
        truth = self.ema.update(truth)

        flags = []
        if tremor_idx > CFG.TREMOR_THRESHOLD:
            flags.append(f"VOICE TREMOR DETECTED: {tremor_idx:.3f} (threshold: {CFG.TREMOR_THRESHOLD})")
        if f0_var > 0.15:
            flags.append(f"PITCH INSTABILITY: variance {f0_var:.3f}")
        if jitter > 0.30:
            flags.append(f"HIGH JITTER: {jitter:.3f}")

        return ChannelScore(
            name="Voice Stress",
            truth_score=clamp(truth),
            deception_risk=clamp(100 - truth),
            confidence=0.90 if rms_energy > 0.01 else 0.30,
            flags=flags,
            metadata={
                "f0_hz": round(f0, 2),
                "f0_variance": round(f0_var, 4),
                "tremor_index": round(tremor_idx, 4),
                "jitter": round(jitter, 4),
                "shimmer": round(shimmer, 4),
                "rms_energy": round(rms_energy, 4),
                "frequency_bands": {k: round(v, 4) for k, v in bands.items()},
                "segments_analyzed": self.segment_count,
            }
        )

    def generate_synthetic_audio(self, duration: float = 1.0,
                                 stress_level: float = 0.5,
                                 f0_hz: float = 150.0) -> np.ndarray:
        """
        Generate synthetic voice signal with controllable stress level.
        stress_level: 0.0 = calm, 1.0 = highly stressed/deceptive
        """
        t = np.linspace(0, duration, int(self.sr * duration))
        # Base voice fundamental
        voice = np.sin(2 * np.pi * f0_hz * t)
        # Add harmonics
        for h in [2, 3, 4, 5]:
            voice += (1/h) * np.sin(2 * np.pi * f0_hz * h * t)
        # Add stress-proportional tremor (8–12 Hz AM modulation)
        tremor_freq = 10.0
        tremor_mod  = 1 + stress_level * 0.4 * np.sin(2 * np.pi * tremor_freq * t)
        voice *= tremor_mod
        # Add F0 jitter (pitch variance)
        f0_noise = np.cumsum(np.random.randn(len(t)) * stress_level * 0.5)
        voice += 0.1 * stress_level * np.sin(2 * np.pi * (f0_hz + f0_noise) * t)
        # Add background noise
        noise = np.random.randn(len(t)) * 0.05 * (1 + stress_level)
        voice = voice + noise
        # Normalize
        return (voice / (np.max(np.abs(voice)) + 1e-9)).astype(np.float32)

    def analyze_synthetic(self, n_segments: int = 50,
                           stress_level: float = 0.5) -> List[ChannelScore]:
        """Run synthetic stress analysis for demo."""
        scores = []
        for _ in range(n_segments):
            audio = self.generate_synthetic_audio(
                duration=0.5,
                stress_level=stress_level + random.gauss(0, 0.1),
                f0_hz=120 + random.gauss(0, 20)
            )
            scores.append(self.analyze_segment(audio))
        return scores


voice_analyzer = VoiceStressAnalyzer(sample_rate=CFG.SAMPLE_RATE)

# Test with synthetic audio
test_audio  = voice_analyzer.generate_synthetic_audio(duration=1.0, stress_level=0.3)
test_voice  = voice_analyzer.analyze_segment(test_audio)

print("✅ VoiceStressAnalyzer ready")
print(f"   Sample Rate         : {CFG.SAMPLE_RATE} Hz")
print(f"   Frequency Bands     : {len(VoiceStressAnalyzer.STRESS_BANDS)}")
print(f"   Test F0             : {test_voice.metadata['f0_hz']} Hz")
print(f"   Test Tremor Index   : {test_voice.metadata['tremor_index']}")
print(f"   Test truth score    : {test_voice.truth_score:.1f}%")

---
<a id="7-nlp-engine"></a>
## 🧠 Module 4 — NLP Deception Engine

Analyzes speech content for linguistic deception markers. Hedge words, protest phrases, filler words, and lack of specific detail are all documented deception signals.

In [ ]:
# ============================================================
# NLP LINGUISTIC DECEPTION ENGINE
# ============================================================

class NLPDeceptionEngine:
    """
    Detects deceptive language patterns via:
    - Hedge word frequency (reduces commitment, signals uncertainty)
    - Protest phrases (over-assertions of honesty = deception marker)
    - Filler word density (cognitive load indicator)
    - Negation pattern analysis
    - Specificity scoring (dates, numbers, proper nouns = truth signals)
    - Sentence complexity analysis
    - First-person pronoun distancing (reduced 'I' = distancing from lie)
    - Embedded clauses / over-elaboration detection
    """

    # ─── Lexicons ─────────────────────────────────────────────
    HEDGE_WORDS = {
        "maybe", "perhaps", "possibly", "probably", "might", "could",
        "sort of", "kind of", "i think", "i believe", "i suppose",
        "as far as i know", "to my knowledge", "roughly", "approximately",
        "somewhat", "virtually", "basically", "generally", "usually",
        "normally", "like", "you know", "i guess", "i imagine", "seemingly",
        "apparently", "it seems", "it appears", "more or less", "in a way",
    }

    PROTEST_PHRASES = {
        "trust me", "i swear", "i promise", "believe me", "honestly",
        "i would never", "to tell the truth", "the truth is",
        "frankly speaking", "let me be clear", "as god is my witness",
        "i'm telling you", "you have to believe", "why would i lie",
        "i'm not lying", "i have nothing to hide", "i'm being honest",
        "swear to god", "on my mother's grave", "hand on heart",
    }

    FILLER_WORDS = {
        "um", "uh", "er", "hmm", "ah", "like", "you know", "right",
        "okay", "well", "so", "anyway", "basically", "literally",
        "actually", "obviously", "clearly", "definitely",
    }

    NEGATION_PATTERNS = {
        "didn't", "don't", "won't", "can't", "couldn't", "wouldn't",
        "never", "not", "no one", "nobody", "nothing", "nowhere",
        "nor", "neither", "hardly", "scarcely", "barely",
    }

    FIRST_PERSON_SINGULAR = {"i", "me", "my", "mine", "myself", "i'm", "i've", "i'll"}
    FIRST_PERSON_PLURAL   = {"we", "us", "our", "ours", "ourselves"}

    DATE_PATTERNS = {
        "monday", "tuesday", "wednesday", "thursday", "friday",
        "saturday", "sunday", "january", "february", "march", "april",
        "may", "june", "july", "august", "september", "october",
        "november", "december", "yesterday", "last week", "this morning",
        "last night", "last year", "two days ago",
    }

    def __init__(self):
        self.analysis_history = []

    def tokenize(self, text: str) -> List[str]:
        """Simple whitespace + punctuation tokenizer."""
        import re
        text = text.lower().strip()
        tokens = re.findall(r"\b[\w']+\b", text)
        return tokens

    def count_pattern(self, text: str, patterns: set) -> Tuple[int, List[str]]:
        """Count occurrences of pattern set in text. Returns (count, matched)."""
        text_lower = text.lower()
        matched = []
        for pattern in patterns:
            if pattern in text_lower:
                count = text_lower.count(pattern)
                matched.extend([pattern] * count)
        return len(matched), matched

    def specificity_score(self, text: str, tokens: List[str]) -> float:
        """
        Score how specific/detailed the statement is.
        High specificity → truth signal.
        """
        import re
        score = 0.0
        # Numbers (quantity specificity)
        numbers = re.findall(r'\b\d+\.?\d*\b', text)
        score += len(numbers) * CFG.NUMBER_TRUTH_WEIGHT
        # Date references
        date_count, _ = self.count_pattern(text, self.DATE_PATTERNS)
        score += date_count * CFG.DATE_TRUTH_WEIGHT
        # Proper nouns (words with initial caps, crude proxy)
        proper_nouns = [t for t in text.split() if t and t[0].isupper() and len(t) > 2]
        score += len(proper_nouns) * 1.5
        # Named locations / places (very crude — would use NER in production)
        location_words = {"at", "in", "near", "from", "to", "on"}
        location_context = sum(1 for t in tokens if t in location_words)
        score += location_context * 1.0
        return score

    def analyze(self, text: str) -> ChannelScore:
        """
        Full NLP deception analysis of input text.
        Returns ChannelScore with truth_score, flags, and detailed metadata.
        """
        tokens    = self.tokenize(text)
        word_count = len(tokens)

        if word_count == 0:
            return ChannelScore("NLP", 50, 50, 0.0, ["NO TEXT PROVIDED"])

        # ─── Count Patterns ───────────────────────────────────
        hedge_n,   hedge_matches   = self.count_pattern(text, self.HEDGE_WORDS)
        protest_n, protest_matches = self.count_pattern(text, self.PROTEST_PHRASES)
        filler_n,  filler_matches  = self.count_pattern(text, self.FILLER_WORDS)
        neg_n,     neg_matches     = self.count_pattern(text, self.NEGATION_PATTERNS)

        # ─── Pronoun Analysis ─────────────────────────────────
        i_count  = sum(1 for t in tokens if t in self.FIRST_PERSON_SINGULAR)
        we_count = sum(1 for t in tokens if t in self.FIRST_PERSON_PLURAL)
        i_ratio  = i_count / word_count if word_count > 0 else 0
        # Reduced 'I' usage = psychological distancing = deception signal
        i_distancing = i_ratio < 0.04 and word_count > 15

        # ─── Specificity ───────────────────────────────────────
        specificity = self.specificity_score(text, tokens)

        # ─── Sentence Analysis ────────────────────────────────
        sentences  = [s.strip() for s in text.replace("!", ".").replace("?", ".").split(".") if s.strip()]
        n_sents    = max(1, len(sentences))
        avg_sent_len = word_count / n_sents

        # ─── Deception Score Computation ─────────────────────
        deception_score = 50.0  # neutral baseline

        # Penalty: deception indicators
        deception_score += hedge_n   * CFG.HEDGE_RISK_WEIGHT
        deception_score += protest_n * CFG.PROTEST_RISK_WEIGHT
        deception_score += filler_n  * CFG.FILLER_RISK_WEIGHT
        deception_score += neg_n     * CFG.NEGATION_RISK_WEIGHT
        if avg_sent_len > 30: deception_score += 8   # over-elaboration
        if word_count < 8:    deception_score += 12  # evasive brevity
        if i_distancing:      deception_score += 10  # psychological distancing

        # Reward: truth indicators
        deception_score -= specificity
        if i_ratio > 0.08:    deception_score -= 5   # high I-usage = ownership
        if avg_sent_len < 20: deception_score -= 3   # concise and direct

        deception_score = clamp(deception_score)
        truth_score     = clamp(100 - deception_score)

        # ─── Flags ────────────────────────────────────────────
        flags = []
        if protest_n > 0:
            flags.append(f"PROTEST LANGUAGE: {', '.join(set(protest_matches))}")
        if hedge_n > 3:
            flags.append(f"EXCESSIVE HEDGING: {hedge_n} instances")
        if filler_n > 5:
            flags.append(f"HIGH FILLER DENSITY: {filler_n} fillers in {word_count} words")
        if word_count < 8:
            flags.append("EVASIVE BREVITY: response too short")
        if i_distancing:
            flags.append("PRONOUN DISTANCING: reduced first-person usage")
        if avg_sent_len > 30:
            flags.append(f"OVER-ELABORATION: avg sentence {avg_sent_len:.0f} words")

        result = ChannelScore(
            name="NLP Analysis",
            truth_score=truth_score,
            deception_risk=deception_score,
            confidence=min(0.95, 0.50 + word_count * 0.01),
            flags=flags,
            metadata={
                "word_count": word_count,
                "sentence_count": n_sents,
                "avg_sentence_length": round(avg_sent_len, 1),
                "hedge_count": hedge_n,
                "hedge_words": list(set(hedge_matches))[:5],
                "protest_count": protest_n,
                "protest_phrases": list(set(protest_matches))[:3],
                "filler_count": filler_n,
                "negation_count": neg_n,
                "specificity_score": round(specificity, 2),
                "i_ratio": round(i_ratio, 3),
                "i_distancing": i_distancing,
                "deception_score_raw": round(deception_score, 2),
            }
        )

        self.analysis_history.append(result)
        return result


# ─── Test ─────────────────────────────────────────────────────
nlp_engine = NLPDeceptionEngine()

# Test Case 1: Deceptive statement
deceptive_text = (
    "I swear I wasn't there that evening, you have to believe me. "
    "I would never do something like that, honestly. Maybe someone "
    "else did it, I don't know, like I said I wasn't anywhere near."
)

# Test Case 2: Truthful statement
truthful_text = (
    "I was at the office at 3pm on Tuesday the 14th. I had a meeting "
    "with Sarah and James from accounting. We finished at around 4:30 "
    "and I drove home on Main Street, arriving at approximately 5:15."
)

r1 = nlp_engine.analyze(deceptive_text)
r2 = nlp_engine.analyze(truthful_text)

print("✅ NLPDeceptionEngine ready")
print(f"\n  Lexicon sizes:")
print(f"    Hedge words     : {len(NLPDeceptionEngine.HEDGE_WORDS)}")
print(f"    Protest phrases : {len(NLPDeceptionEngine.PROTEST_PHRASES)}")
print(f"    Filler words    : {len(NLPDeceptionEngine.FILLER_WORDS)}")

print(f"\n  Test — Deceptive Statement:")
print(f"    Truth Score     : {r1.truth_score:.1f}%")
print(f"    Verdict         : {r1.label}")
print(f"    Protest count   : {r1.metadata['protest_count']}")
print(f"    Hedge count     : {r1.metadata['hedge_count']}")
print(f"    Flags           : {len(r1.flags)}")

print(f"\n  Test — Truthful Statement:")
print(f"    Truth Score     : {r2.truth_score:.1f}%")
print(f"    Verdict         : {r2.label}")
print(f"    Specificity     : {r2.metadata['specificity_score']}")
print(f"    Flags           : {len(r2.flags)}")

---
<a id="8-response-delay"></a>
## ⏱ Module 5 — Response Delay & Cognitive Load Tracker

In [ ]:
# ============================================================
# RESPONSE DELAY & COGNITIVE LOAD TRACKER
# ============================================================

class ResponseDelayTracker:
    """
    Measures and interprets the time between question and response onset.

    Cognitive Load Theory: Deceptive responses require real-time story
    construction, monitoring of the listener, and managing multiple
    versions of events — creating measurable latency spikes.

    Also tracks:
    - Inter-word pause duration (micro-hesitations)
    - Speech rate (words per minute)
    - Pause-to-speech ratio
    """

    def __init__(self):
        self.response_times  = deque(maxlen=50)
        self.pause_durations = deque(maxlen=100)
        self.speech_rates    = deque(maxlen=50)
        self.question_time   = None
        self.ema             = EMA(alpha=0.20)

    def mark_question(self):
        """Call this when a question is posed."""
        self.question_time = time.time()

    def mark_response(self, text: str = "") -> float:
        """Call this when response begins. Returns latency in seconds."""
        if self.question_time is None:
            return 0.0
        latency = time.time() - self.question_time
        self.response_times.append(latency)
        self.question_time = None

        if text:
            words = len(text.split())
            wpm   = (words / latency) * 60 if latency > 0 else 0
            self.speech_rates.append(wpm)

        return latency

    def analyze_latency(self, latency_seconds: float,
                         word_count: int = 0) -> ChannelScore:
        """Score a given response latency."""
        self.response_times.append(latency_seconds)
        avg_lat = np.mean(self.response_times)

        # ─── Score from latency ───────────────────────────────
        if latency_seconds < CFG.FAST_RESPONSE:
            truth = 95.0   # spontaneous
            latency_label = "SPONTANEOUS"
        elif latency_seconds < CFG.NORMAL_RESPONSE:
            truth = 80.0   # normal
            latency_label = "NORMAL"
        elif latency_seconds < CFG.SLOW_RESPONSE:
            truth = 60.0   # slightly elevated
            latency_label = "ELEVATED"
        elif latency_seconds < CFG.HIGH_LATENCY:
            truth = 35.0   # high cognitive load
            latency_label = "HIGH LATENCY"
        else:
            truth = 15.0   # very high — strong deception indicator
            latency_label = "EXTREME LATENCY"

        # Bonus: if response is rapid but very short (deflection)
        if latency_seconds < CFG.FAST_RESPONSE and word_count < 5:
            truth -= 20
            latency_label = "RAPID DEFLECTION"

        truth = self.ema.update(clamp(truth))

        flags = []
        if latency_seconds > CFG.HIGH_LATENCY:
            flags.append(f"EXTREME RESPONSE DELAY: {latency_seconds:.2f}s")
        elif latency_seconds > CFG.SLOW_RESPONSE:
            flags.append(f"ELEVATED RESPONSE LATENCY: {latency_seconds:.2f}s")
        if latency_label == "RAPID DEFLECTION":
            flags.append("RAPID SHORT DEFLECTION DETECTED")

        return ChannelScore(
            name="Response Delay",
            truth_score=clamp(truth),
            deception_risk=clamp(100 - truth),
            confidence=0.75,
            flags=flags,
            metadata={
                "latency_seconds": round(latency_seconds, 3),
                "latency_label": latency_label,
                "average_latency": round(avg_lat, 3),
                "n_responses": len(self.response_times),
                "thresholds": {
                    "fast": CFG.FAST_RESPONSE,
                    "normal": CFG.NORMAL_RESPONSE,
                    "slow": CFG.SLOW_RESPONSE,
                    "high": CFG.HIGH_LATENCY,
                }
            }
        )

    def analyze_synthetic(self, n: int = 30,
                           stress_level: float = 0.5) -> List[ChannelScore]:
        """Simulate response delays with varying stress."""
        scores = []
        for _ in range(n):
            # Stressed subjects have longer, more variable latencies
            base_latency = 0.3 + stress_level * 0.8
            latency = base_latency + random.expovariate(1.0 / (0.3 + stress_level))
            latency = min(latency, 4.0)
            scores.append(self.analyze_latency(latency, word_count=random.randint(5, 30)))
        return scores


delay_tracker = ResponseDelayTracker()

# Test examples
print("✅ ResponseDelayTracker ready")
print("\n  Latency Classification:")
for lat in [0.2, 0.5, 0.85, 1.2, 2.0]:
    s = delay_tracker.analyze_latency(lat)
    print(f"    {lat:.1f}s → {s.metadata['latency_label']:20s} → Truth: {s.truth_score:.0f}%")

---
<a id="9-physiological"></a>
## 💓 Module 6 — Physiological Signal Estimator

In [ ]:
# ============================================================
# PHYSIOLOGICAL SIGNAL ESTIMATOR (CV-derived)
# ============================================================

class PhysiologicalEstimator:
    """
    Estimates physiological arousal from video frames:

    1. Remote Photoplethysmography (rPPG) — estimates heart rate
       from subtle skin color changes (RGB variation in face ROI)
    2. Facial Flush Detection — increased redness from blood flow
    3. Perspiration Estimation — skin texture changes under stress
    4. Micro-tremor in face region — involuntary movement amplitude

    Note: These are approximations. Clinical systems use dedicated
    hardware (galvanic skin response, respiration belt, etc.)
    """

    def __init__(self):
        self.face_cascade  = cv2.CascadeClassifier(
            cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
        )
        self.rgb_history   = deque(maxlen=150)   # ~5 seconds at 30fps
        self.flush_history = deque(maxlen=60)
        self.ema           = EMA(alpha=0.10)
        self.frame_count   = 0

    def extract_face_roi(self, frame: np.ndarray) -> Optional[np.ndarray]:
        """Extract forehead ROI for rPPG signal."""
        gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(60, 60))
        if len(faces) == 0:
            return None
        x, y, w, h = max(faces, key=lambda f: f[2]*f[3])
        # Forehead region (upper 1/4 of face)
        forehead_y1 = y + int(h * 0.05)
        forehead_y2 = y + int(h * 0.30)
        forehead_x1 = x + int(w * 0.15)
        forehead_x2 = x + int(w * 0.85)
        return frame[forehead_y1:forehead_y2, forehead_x1:forehead_x2]

    def estimate_rppg(self, roi: np.ndarray) -> Tuple[float, float]:
        """
        Estimate heart rate from mean RGB of face ROI.
        Returns (estimated_hr_bpm, confidence).
        """
        if roi is None or roi.size == 0:
            return 72.0, 0.1
        mean_rgb = roi.mean(axis=(0, 1))  # [B, G, R]
        self.rgb_history.append(mean_rgb)

        if len(self.rgb_history) < 30:
            return 72.0, 0.2

        rgb_arr = np.array(self.rgb_history)
        # Green channel has strongest blood volume pulse signal
        green_channel = rgb_arr[:, 1]
        # Remove DC component
        green_ac = green_channel - np.mean(green_channel)
        # FFT to find dominant frequency
        n    = len(green_ac)
        freqs = np.abs(fftfreq(n, d=1/30))[:n//2]   # assume 30fps
        power = np.abs(fft(green_ac))[:n//2]
        # HR range: 50–150 bpm = 0.83–2.5 Hz
        hr_mask   = (freqs >= 0.83) & (freqs <= 2.5)
        if not hr_mask.any():
            return 72.0, 0.2
        peak_freq = freqs[hr_mask][np.argmax(power[hr_mask])]
        hr_bpm    = peak_freq * 60
        # Confidence from spectral peak sharpness
        confidence = min(0.85, float(np.max(power[hr_mask]) / (np.mean(power) + 1e-9)) * 0.1)
        return float(hr_bpm), confidence

    def estimate_flush(self, roi: np.ndarray) -> float:
        """Estimate redness (flush) as ratio of R channel to mean."""
        if roi is None or roi.size == 0:
            return 0.0
        mean_r = float(roi[:, :, 2].mean())
        mean_g = float(roi[:, :, 1].mean())
        mean_b = float(roi[:, :, 0].mean())
        # Flush = elevated red relative to blue/green
        flush = (mean_r - (mean_g + mean_b) / 2) / 255.0
        self.flush_history.append(flush)
        return clamp(flush * 100, 0, 100)

    def analyze_frame(self, frame: np.ndarray) -> ChannelScore:
        """Extract physiological signals from one frame."""
        self.frame_count += 1
        roi = self.extract_face_roi(frame)

        hr_bpm, hr_confidence = self.estimate_rppg(roi)
        flush_pct = self.estimate_flush(roi)

        # Simulate perspiration index (texture changes — crude proxy)
        persp_idx = abs(flush_pct - 30) * 0.5 + random.gauss(10, 5)
        persp_idx = clamp(persp_idx)

        # ─── Score Computation ────────────────────────────────
        # HR component: elevated HR = stress
        if 60 <= hr_bpm <= 90:
            hr_trust = 90
        elif 50 <= hr_bpm < 60 or 90 < hr_bpm <= 100:
            hr_trust = 70
        elif hr_bpm > 100:
            hr_trust = max(0, 90 - (hr_bpm - 100) * 2)
        else:
            hr_trust = 60

        # Flush component
        flush_trust = clamp(100 - flush_pct * 1.2)

        # Perspiration component
        persp_trust = clamp(100 - persp_idx)

        truth = (hr_trust * 0.45 + flush_trust * 0.35 + persp_trust * 0.20)
        truth = self.ema.update(clamp(truth))

        flags = []
        if hr_bpm > 100:
            flags.append(f"ELEVATED HEART RATE: ~{hr_bpm:.0f} bpm")
        if flush_pct > 40:
            flags.append(f"FACIAL FLUSH DETECTED: {flush_pct:.0f}%")

        return ChannelScore(
            name="Physiological",
            truth_score=clamp(truth),
            deception_risk=clamp(100 - truth),
            confidence=hr_confidence,
            flags=flags,
            metadata={
                "estimated_hr_bpm": round(hr_bpm, 1),
                "facial_flush_pct": round(flush_pct, 1),
                "perspiration_idx": round(persp_idx, 1),
                "roi_detected": roi is not None,
                "frames_analyzed": self.frame_count,
            }
        )

    def analyze_synthetic(self, n: int = 50, stress_level: float = 0.5) -> List[ChannelScore]:
        scores = []
        for i in range(n):
            frame = np.zeros((480, 640, 3), dtype=np.uint8)
            # Simulate face region with stress-based redness
            base_r = int(160 + stress_level * 60 + random.gauss(0, 10))
            base_g = int(130 + random.gauss(0, 8))
            base_b = int(120 + random.gauss(0, 8))
            cv2.ellipse(frame, (320, 240), (100, 130), 0, 0, 360,
                        (base_b, base_g, min(255, base_r)), -1)
            scores.append(self.analyze_frame(frame))
        return scores


physio_estimator = PhysiologicalEstimator()
test_physio_frame = np.random.randint(100, 200, (480, 640, 3), dtype=np.uint8)
test_physio = physio_estimator.analyze_frame(test_physio_frame)

print("✅ PhysiologicalEstimator ready")
print(f"   Estimated HR        : {test_physio.metadata['estimated_hr_bpm']} bpm")
print(f"   Facial Flush        : {test_physio.metadata['facial_flush_pct']}%")
print(f"   Test truth score    : {test_physio.truth_score:.1f}%")

---
<a id="10-score-fusion"></a>
## 🔀 Module 7 — Score Fusion Engine

Fuses all 6 channel scores using weighted average + anomaly detection + temporal smoothing.

In [ ]:
# ============================================================
# SCORE FUSION ENGINE
# ============================================================

class ScoreFusionEngine:
    """
    Fuses scores from all 6 analysis channels into a single
    Truthfulness Confidence Score using:

    1. Weighted average fusion
    2. Confidence-adjusted weighting (low-confidence channels penalized)
    3. Cross-modal consistency check (conflicting channels trigger warning)
    4. Temporal exponential moving average (EMA) smoothing
    5. Anomaly detection via Isolation Forest (optional)
    """

    CHANNEL_WEIGHTS = {
        "Micro-Expression": CFG.WEIGHT_MICRO_EXPR,
        "Eye Movement":     CFG.WEIGHT_EYE_MOVE,
        "Voice Stress":     CFG.WEIGHT_VOICE_STRESS,
        "NLP Analysis":     CFG.WEIGHT_NLP,
        "Response Delay":   CFG.WEIGHT_DELAY,
        "Physiological":    CFG.WEIGHT_PHYSIO,
    }

    def __init__(self):
        self.ema            = EMA(alpha=CFG.EMA_ALPHA)
        self.score_history  = deque(maxlen=CFG.TIMELINE_BUFFER)
        self.channel_history = {k: deque(maxlen=50) for k in self.CHANNEL_WEIGHTS}
        self.event_log      = []
        self.session_start  = time.time()

    def fuse(self, channels: Dict[str, ChannelScore]) -> Tuple[float, List[str]]:
        """
        Fuse channel scores into composite truth score.
        Returns (truth_score 0–100, list_of_flags).
        """
        total_weight = 0.0
        weighted_sum = 0.0
        all_flags    = []

        for name, base_weight in self.CHANNEL_WEIGHTS.items():
            if name not in channels:
                continue
            cs = channels[name]
            # Confidence-adjusted weight
            adj_weight = base_weight * max(0.1, cs.confidence)
            weighted_sum += cs.truth_score * adj_weight
            total_weight += adj_weight
            all_flags.extend(cs.flags)
            self.channel_history[name].append(cs.truth_score)

        raw_score = weighted_sum / total_weight if total_weight > 0 else 50.0

        # Cross-modal consistency check
        scores_list = [channels[n].truth_score for n in channels]
        if len(scores_list) >= 2:
            spread = max(scores_list) - min(scores_list)
            if spread > 40:
                all_flags.append(f"CROSS-MODAL INCONSISTENCY: {spread:.0f}pt spread between channels")

        # Temporal smoothing
        smoothed = self.ema.update(raw_score)
        self.score_history.append(smoothed)

        return clamp(smoothed), all_flags

    def build_result(self, channels: Dict[str, ChannelScore],
                     raw_text: str = "",
                     duration: float = 0.0) -> AnalysisResult:
        """Build complete AnalysisResult from channel scores."""
        truth_score, flags = self.fuse(channels)
        verdict = verdict_from_score(truth_score)

        return AnalysisResult(
            session_id=generate_session_id(),
            timestamp=datetime.datetime.now(),
            duration_seconds=duration or (time.time() - self.session_start),
            channels=channels,
            truth_score=truth_score,
            deception_risk=clamp(100 - truth_score),
            verdict=verdict,
            flags=list(set(flags)),
            timeline=list(self.score_history),
            raw_text=raw_text,
        )

    def log_event(self, event_type: str, message: str, score: float = None):
        """Add to behavioral event log."""
        self.event_log.append({
            "time": round(time.time() - self.session_start, 2),
            "type": event_type,
            "message": message,
            "score": score,
        })


fusion_engine = ScoreFusionEngine()
print("✅ ScoreFusionEngine ready")
print(f"   Channel Weights:")
for name, w in ScoreFusionEngine.CHANNEL_WEIGHTS.items():
    bar = "█" * int(w * 40)
    print(f"    {name:20s} {bar:10s} {w:.0%}")

---
<a id="11-visualization"></a>
## 📊 Module 8 — Visualization Dashboard

Full matplotlib dashboard displaying all analysis channels, truth score history, and verdict.

In [ ]:
# ============================================================
# VISUALIZATION DASHBOARD
# ============================================================

class VeritasDashboard:
    """
    Renders a comprehensive multi-panel analysis dashboard using matplotlib.
    Panels:
      [1] Truth Score Gauge (ring chart)
      [2] Channel Score Bar Chart
      [3] Timeline History
      [4] Frequency Band Spectrum (voice)
      [5] NLP Linguistic Breakdown
      [6] Eye Movement Heatmap
      [7] Deception Flag Summary
      [8] Session Stats Table
    """

    def __init__(self):
        self.CMAP = LinearSegmentedColormap.from_list(
            "veritas", ["#ff3c6e", "#ffb800", "#00d4ff", "#39ff14"], N=256
        )

    def score_color(self, score: float) -> str:
        return color_from_score(score)

    def render(self, result: AnalysisResult, figsize=(20, 14)) -> plt.Figure:
        """Render the complete dashboard figure."""
        fig = plt.figure(figsize=figsize, facecolor="#030812")
        fig.suptitle(
            "VERITAS — DECEPTION ANALYSIS REPORT",
            fontsize=16, fontweight="bold",
            color=CYAN, y=0.98, fontfamily="monospace", letterspacing=3
        )

        gs = gridspec.GridSpec(
            3, 4, figure=fig,
            hspace=0.45, wspace=0.35,
            top=0.93, bottom=0.06, left=0.06, right=0.97
        )

        # ─ Panel 1: Truth Gauge ─────────────────────────────
        ax1 = fig.add_subplot(gs[0, 0], polar=True)
        self._draw_gauge(ax1, result.truth_score, result.verdict)

        # ─ Panel 2: Channel Scores ───────────────────────────
        ax2 = fig.add_subplot(gs[0, 1:3])
        self._draw_channel_bars(ax2, result.channels)

        # ─ Panel 3: Session Info ─────────────────────────────
        ax3 = fig.add_subplot(gs[0, 3])
        self._draw_session_info(ax3, result)

        # ─ Panel 4: Timeline ─────────────────────────────────
        ax4 = fig.add_subplot(gs[1, :2])
        self._draw_timeline(ax4, result.timeline)

        # ─ Panel 5: Voice Spectrum ───────────────────────────
        ax5 = fig.add_subplot(gs[1, 2])
        voice = result.channels.get("Voice Stress")
        if voice:
            self._draw_voice_spectrum(ax5, voice)

        # ─ Panel 6: NLP Breakdown ────────────────────────────
        ax6 = fig.add_subplot(gs[1, 3])
        nlp = result.channels.get("NLP Analysis")
        if nlp:
            self._draw_nlp_breakdown(ax6, nlp)

        # ─ Panel 7: Deception Flags ──────────────────────────
        ax7 = fig.add_subplot(gs[2, :2])
        self._draw_flags(ax7, result.flags)

        # ─ Panel 8: Confidence Heatmap ───────────────────────
        ax8 = fig.add_subplot(gs[2, 2:])
        self._draw_confidence_heatmap(ax8, result.channels)

        return fig

    def _draw_gauge(self, ax, score: float, verdict: str):
        """Semi-circular truth gauge."""
        ax.set_facecolor("#080f1e")
        score_c = clamp(score)
        # Background arc
        theta_bg = np.linspace(np.pi, 0, 200)
        ax.plot(theta_bg, [1]*200, color=DIM, linewidth=18, alpha=0.3)
        # Score arc
        angle_end = np.pi - (score_c / 100) * np.pi
        theta_sc  = np.linspace(np.pi, angle_end, 200)
        sc_color  = self.score_color(score_c)
        ax.plot(theta_sc, [1]*200, color=sc_color, linewidth=18, alpha=0.9, solid_capstyle="round")
        # Score text
        ax.text(np.pi/2, 0.2, f"{score_c:.0f}%",
                ha="center", va="center", fontsize=28, fontweight="bold",
                color=sc_color, fontfamily="monospace")
        ax.text(np.pi/2, -0.15, verdict.replace("✅ ","").replace("🔵 ","").replace("🟡 ","").replace("🟠 ","").replace("🔴 ",""),
                ha="center", va="center", fontsize=9,
                color=sc_color, fontfamily="monospace")
        ax.text(np.pi/2, -0.42, "TRUTH SCORE",
                ha="center", va="center", fontsize=7,
                color=DIM, fontfamily="monospace")
        ax.set_ylim(-0.6, 1.3)
        ax.axis("off")
        ax.set_title("CONFIDENCE GAUGE", color=CYAN, fontsize=9,
                     pad=6, fontfamily="monospace")

    def _draw_channel_bars(self, ax, channels: Dict[str, ChannelScore]):
        """Horizontal bar chart of all channel truth scores."""
        ax.set_facecolor("#080f1e")
        names  = list(channels.keys())
        scores = [channels[n].truth_score for n in names]
        confs  = [channels[n].confidence  for n in names]
        colors = [self.score_color(s) for s in scores]

        y_pos = range(len(names))
        bars  = ax.barh(y_pos, scores, color=colors, alpha=0.75, height=0.55,
                        edgecolor="none")
        # Confidence indicator
        for i, (s, c) in enumerate(zip(scores, confs)):
            ax.barh(i, s * c, color="white", alpha=0.12, height=0.55)
            ax.text(s + 1, i, f"{s:.0f}%", va="center", fontsize=9,
                    color=colors[i], fontfamily="monospace", fontweight="bold")
            ax.text(-1, i, f"{c:.0%}", va="center", ha="right", fontsize=7,
                    color=DIM, fontfamily="monospace")

        ax.set_yticks(list(y_pos))
        ax.set_yticklabels(names, fontsize=9, fontfamily="monospace")
        ax.set_xlim(-5, 115)
        ax.set_xlabel("Truth Score (%)", color=DIM, fontsize=8)
        ax.tick_params(colors=DIM, labelsize=8)
        ax.axvline(50, color=YELLOW, linewidth=0.7, linestyle="--", alpha=0.5)
        ax.axvline(75, color=GREEN,  linewidth=0.7, linestyle="--", alpha=0.5)
        ax.grid(axis="x", alpha=0.2)
        ax.set_title("CHANNEL TRUTH SCORES  (opacity = confidence)", color=CYAN,
                     fontsize=9, fontfamily="monospace")

    def _draw_session_info(self, ax, result: AnalysisResult):
        """Session metadata text panel."""
        ax.set_facecolor("#080f1e")
        ax.axis("off")
        info_lines = [
            ("SESSION ID",  result.session_id),
            ("TIMESTAMP",   result.timestamp.strftime("%H:%M:%S")),
            ("DURATION",    f"{result.duration_seconds:.1f}s"),
            ("VERDICT",     result.verdict.split(" ", 1)[-1]),
            ("TRUTH SCORE", f"{result.truth_score:.1f}%"),
            ("RISK LEVEL",  f"{result.deception_risk:.1f}%"),
            ("FLAGS",       str(len(result.flags))),
            ("CHANNELS",    str(len(result.channels))),
        ]
        for i, (label, value) in enumerate(info_lines):
            y = 0.95 - i * 0.115
            ax.text(0.02, y, label + ":", transform=ax.transAxes,
                    fontsize=7.5, color=DIM, fontfamily="monospace")
            ax.text(0.98, y, value, transform=ax.transAxes,
                    fontsize=8, color=CYAN, fontfamily="monospace",
                    ha="right", fontweight="bold")
            ax.axhline(y - 0.045, xmin=0, xmax=1, color="#0d2044",
                       linewidth=0.5, transform=ax.transAxes)
        ax.set_title("SESSION METADATA", color=CYAN, fontsize=9, fontfamily="monospace")

    def _draw_timeline(self, ax, timeline: List[float]):
        """Truth score over time with shaded regions."""
        ax.set_facecolor("#080f1e")
        if not timeline:
            ax.text(0.5, 0.5, "NO DATA", ha="center", va="center",
                    color=DIM, transform=ax.transAxes, fontfamily="monospace")
            ax.set_title("TRUTH SCORE TIMELINE", color=CYAN, fontsize=9, fontfamily="monospace")
            return

        x = np.arange(len(timeline))
        y = np.array(timeline)
        # Gradient fill
        ax.fill_between(x, 0, y, where=y>=75, alpha=0.15, color=GREEN,    interpolate=True)
        ax.fill_between(x, 0, y, where=(y>=55)&(y<75), alpha=0.15, color=CYAN,   interpolate=True)
        ax.fill_between(x, 0, y, where=(y>=40)&(y<55), alpha=0.15, color=YELLOW, interpolate=True)
        ax.fill_between(x, 0, y, where=y<40,  alpha=0.15, color=RED,     interpolate=True)
        # Main line
        ax.plot(x, y, color=CYAN, linewidth=1.5, alpha=0.9)
        # Threshold lines
        for thresh, color, label in [(75, GREEN, "TRUTHFUL"), (55, CYAN, "LIKELY T."),
                                      (40, YELLOW, "INCON."), (25, RED, "DECEPTIVE")]:
            ax.axhline(thresh, color=color, linewidth=0.6, linestyle="--", alpha=0.5)
            ax.text(len(x) + 0.5, thresh, label, fontsize=6, color=color,
                    fontfamily="monospace", va="center")
        ax.set_xlim(0, max(len(x), 1))
        ax.set_ylim(0, 100)
        ax.set_xlabel("Time Steps", color=DIM, fontsize=8)
        ax.set_ylabel("Truth Score %", color=DIM, fontsize=8)
        ax.tick_params(colors=DIM, labelsize=7)
        ax.grid(alpha=0.15)
        ax.set_title("TRUTH SCORE TIMELINE", color=CYAN, fontsize=9, fontfamily="monospace")

    def _draw_voice_spectrum(self, ax, voice_score: ChannelScore):
        """Voice frequency band energy visualization."""
        ax.set_facecolor("#080f1e")
        bands = voice_score.metadata.get("frequency_bands", {})
        if not bands:
            ax.text(0.5, 0.5, "NO AUDIO", ha="center", va="center",
                    color=DIM, transform=ax.transAxes, fontfamily="monospace")
            ax.set_title("VOICE SPECTRUM", color=CYAN, fontsize=9, fontfamily="monospace")
            return
        names  = list(bands.keys())
        values = list(bands.values())
        # Color by stress level (higher sibilance/air = stress)
        colors = [GREEN if i < 2 else YELLOW if i < 3 else RED for i in range(len(names))]
        ax.bar(names, values, color=colors, alpha=0.7, edgecolor="none")
        ax.set_xticklabels([n.replace("_", "\n") for n in names], fontsize=7,
                            fontfamily="monospace", color=DIM)
        ax.tick_params(colors=DIM, labelsize=7)
        ax.set_ylabel("Energy", color=DIM, fontsize=7)
        ax.grid(axis="y", alpha=0.2)
        ax.set_title("VOICE FREQ BANDS", color=CYAN, fontsize=9, fontfamily="monospace")

    def _draw_nlp_breakdown(self, ax, nlp_score: ChannelScore):
        """NLP signal counts as color-coded bar chart."""
        ax.set_facecolor("#080f1e")
        m = nlp_score.metadata
        labels = ["Hedges", "Protests", "Fillers", "Negations", "Specificity"]
        values = [
            m.get("hedge_count", 0),
            m.get("protest_count", 0),
            m.get("filler_count", 0),
            m.get("negation_count", 0),
            min(10, m.get("specificity_score", 0)),
        ]
        colors = [RED, RED, YELLOW, ORANGE, GREEN]
        bars   = ax.bar(labels, values, color=colors, alpha=0.7, edgecolor="none")
        ax.set_xticklabels(labels, fontsize=7, fontfamily="monospace",
                            rotation=20, ha="right", color=DIM)
        ax.tick_params(colors=DIM, labelsize=7)
        ax.set_ylabel("Count / Score", color=DIM, fontsize=7)
        ax.grid(axis="y", alpha=0.2)
        ax.set_title("NLP SIGNAL BREAKDOWN", color=CYAN, fontsize=9, fontfamily="monospace")

    def _draw_flags(self, ax, flags: List[str]):
        """Deception flags as scrollable text."""
        ax.set_facecolor("#080f1e")
        ax.axis("off")
        ax.set_title("DECEPTION FLAGS", color=CYAN, fontsize=9, fontfamily="monospace")
        if not flags:
            ax.text(0.5, 0.5, "✅  NO FLAGS DETECTED — CLEAR",
                    ha="center", va="center", fontsize=10,
                    color=GREEN, fontfamily="monospace", transform=ax.transAxes)
            return
        for i, flag in enumerate(flags[:8]):
            y = 0.88 - i * 0.11
            ax.text(0.01, y, f"⚠  {flag}", transform=ax.transAxes,
                    fontsize=7.5, color=RED, fontfamily="monospace",
                    bbox=dict(boxstyle="round,pad=0.2", facecolor="#1a0a0f",
                              edgecolor="#ff3c6e", alpha=0.6))

    def _draw_confidence_heatmap(self, ax, channels: Dict[str, ChannelScore]):
        """Heatmap of truth scores for each channel over simulated time."""
        ax.set_facecolor("#080f1e")
        ax.set_title("CHANNEL CONFIDENCE MAP", color=CYAN, fontsize=9, fontfamily="monospace")
        names  = list(channels.keys())
        scores = [[channels[n].truth_score] for n in names]
        scores_arr = np.array(scores)
        im = ax.imshow(scores_arr, cmap=self.CMAP, vmin=0, vmax=100,
                       aspect="auto", interpolation="nearest")
        ax.set_yticks(range(len(names)))
        ax.set_yticklabels(names, fontsize=7, fontfamily="monospace")
        ax.set_xticks([])
        ax.tick_params(colors=DIM, labelsize=7)
        for i, n in enumerate(names):
            ax.text(0, i, f"{channels[n].truth_score:.0f}%",
                    ha="center", va="center", fontsize=9,
                    color="white", fontweight="bold", fontfamily="monospace")
        cbar = plt.colorbar(im, ax=ax, shrink=0.8)
        cbar.ax.tick_params(colors=DIM, labelsize=7)
        cbar.set_label("Truth Score %", color=DIM, fontsize=7)


dashboard = VeritasDashboard()
print("✅ VeritasDashboard ready")
print("   Panels: Gauge | Channels | Session | Timeline | Voice | NLP | Flags | Heatmap")

---
<a id="12-full-pipeline"></a>
## 🎬 Module 9 — Full Analysis Pipeline

The `VeritasPipeline` orchestrates all modules together.

In [ ]:
# ============================================================
# VERITAS FULL ANALYSIS PIPELINE
# ============================================================

class VeritasPipeline:
    """
    Orchestrates all 6 analysis modules into a unified pipeline.

    Supports 3 operation modes:
    1. Live camera + microphone (requires device access)
    2. File-based analysis (video file + audio file)
    3. Synthetic simulation (demo mode — no hardware required)
    """

    def __init__(self):
        self.micro_detector  = MicroExpressionDetector()
        self.eye_analyzer    = EyeMovementAnalyzer()
        self.voice_analyzer  = VoiceStressAnalyzer(CFG.SAMPLE_RATE)
        self.nlp_engine      = NLPDeceptionEngine()
        self.delay_tracker   = ResponseDelayTracker()
        self.physio_estimator= PhysiologicalEstimator()
        self.fusion          = ScoreFusionEngine()
        self.dashboard       = VeritasDashboard()
        print("✅ VERITAS Pipeline initialized — all modules online")
        print("   Micro-Expression  |  Eye Movement  |  Voice Stress")
        print("   NLP Engine        |  Delay Tracker |  Physiological")

    def analyze_text(self, text: str,
                     response_latency: float = 0.5) -> AnalysisResult:
        """
        NLP-only analysis for text input.
        For quick analysis of a written statement.
        """
        nlp_score   = self.nlp_engine.analyze(text)
        delay_score = self.delay_tracker.analyze_latency(response_latency,
                                                          word_count=len(text.split()))
        channels = {
            "NLP Analysis":   nlp_score,
            "Response Delay": delay_score,
        }
        return self.fusion.build_result(channels, raw_text=text)

    def analyze_synthetic(self,
                           n_frames: int = 80,
                           n_audio_segments: int = 40,
                           text: str = "",
                           stress_level: float = 0.5,
                           response_latency: float = 0.6,
                           show_progress: bool = True) -> AnalysisResult:
        """
        Full synthetic analysis — runs all 6 channels with
        controllable stress level. Perfect for demo and testing.

        Args:
            n_frames           : Number of video frames to simulate
            n_audio_segments   : Number of audio segments to analyze
            text               : Statement text for NLP analysis
            stress_level       : 0.0 = calm/truthful, 1.0 = highly deceptive
            response_latency   : Simulated response time in seconds
            show_progress      : Show tqdm progress bars
        """
        start = time.time()
        print(f"\n🔬 VERITAS SYNTHETIC ANALYSIS")
        print(f"   Stress Level      : {stress_level:.0%} ({'HIGH DECEPTION' if stress_level > 0.6 else 'MODERATE' if stress_level > 0.3 else 'LOW / TRUTHFUL'})")
        print(f"   Video Frames      : {n_frames}")
        print(f"   Audio Segments    : {n_audio_segments}")
        print(f"   Response Latency  : {response_latency:.2f}s")
        print()

        # ─── Channel 1: Micro-Expressions ──────────────────
        print("  [1/6] Analyzing micro-expressions...")
        micro_scores  = self.micro_detector.analyze_synthetic(n_frames, stress_level)
        avg_micro     = np.mean([s.truth_score for s in micro_scores])
        final_micro   = micro_scores[-1]
        final_micro.truth_score     = avg_micro
        final_micro.deception_risk  = 100 - avg_micro

        # ─── Channel 2: Eye Movement ───────────────────────
        print("  [2/6] Analyzing eye movement & blink rate...")
        eye_scores    = self.eye_analyzer.analyze_synthetic(n_frames, stress_level)
        avg_eye       = np.mean([s.truth_score for s in eye_scores])
        final_eye     = eye_scores[-1]
        final_eye.truth_score    = avg_eye
        final_eye.deception_risk = 100 - avg_eye

        # ─── Channel 3: Voice Stress ───────────────────────
        print("  [3/6] Analyzing voice stress patterns...")
        voice_scores  = self.voice_analyzer.analyze_synthetic(n_audio_segments, stress_level)
        avg_voice     = np.mean([s.truth_score for s in voice_scores])
        final_voice   = voice_scores[-1]
        final_voice.truth_score    = avg_voice
        final_voice.deception_risk = 100 - avg_voice
        # Ensure frequency bands are populated
        if not final_voice.metadata.get("frequency_bands"):
            test_audio = self.voice_analyzer.generate_synthetic_audio(1.0, stress_level)
            bands = self.voice_analyzer.compute_fft_bands(test_audio)
            final_voice.metadata["frequency_bands"] = {k: round(v, 4) for k, v in bands.items()}

        # ─── Channel 4: NLP ────────────────────────────────
        print("  [4/6] Running NLP deception analysis...")
        if not text:
            # Generate a synthetic statement based on stress level
            if stress_level > 0.6:
                text = ("Trust me, I swear I wasn't anywhere near there. "
                        "You have to believe me, I would never do something like that. "
                        "I don't know what you're talking about honestly, "
                        "maybe someone else did it, I'm not sure.")
            elif stress_level > 0.3:
                text = ("I think I was at home, or maybe the office? "
                        "I'm not entirely sure, it's kind of hard to remember. "
                        "Roughly around that time, yes I believe so.")
            else:
                text = ("I was at the office at 3pm on Tuesday the 14th. "
                        "I had a meeting with Sarah from accounting that lasted 90 minutes. "
                        "I drove home on Main Street and arrived at my house at 5:20pm.")
        final_nlp = self.nlp_engine.analyze(text)

        # ─── Channel 5: Response Delay ─────────────────────
        print("  [5/6] Measuring response delay & cognitive load...")
        delay_scores  = self.delay_tracker.analyze_synthetic(20, stress_level)
        avg_delay     = np.mean([s.truth_score for s in delay_scores])
        final_delay   = self.delay_tracker.analyze_latency(
            response_latency + stress_level * 0.5,
            word_count=len(text.split())
        )
        final_delay.truth_score    = (avg_delay + final_delay.truth_score) / 2
        final_delay.deception_risk = 100 - final_delay.truth_score

        # ─── Channel 6: Physiological ──────────────────────
        print("  [6/6] Estimating physiological signals...")
        physio_scores = self.physio_estimator.analyze_synthetic(30, stress_level)
        avg_physio    = np.mean([s.truth_score for s in physio_scores])
        final_physio  = physio_scores[-1]
        final_physio.truth_score    = avg_physio
        final_physio.deception_risk = 100 - avg_physio

        # ─── Build timeline ────────────────────────────────
        print("\n  ⚡ Fusing channels...")
        all_channel_scores = list(zip(micro_scores, eye_scores, voice_scores, physio_scores))
        n_timeline = min(len(micro_scores), len(eye_scores), len(voice_scores))
        for i in range(0, n_timeline, 5):
            frame_channels = {
                "Micro-Expression": micro_scores[i],
                "Eye Movement":     eye_scores[i],
                "Voice Stress":     voice_scores[i],
                "Physiological":    physio_scores[min(i, len(physio_scores)-1)],
                "NLP Analysis":     final_nlp,
                "Response Delay":   final_delay,
            }
            self.fusion.fuse(frame_channels)

        # ─── Final channels ────────────────────────────────
        final_channels = {
            "Micro-Expression": final_micro,
            "Eye Movement":     final_eye,
            "Voice Stress":     final_voice,
            "NLP Analysis":     final_nlp,
            "Response Delay":   final_delay,
            "Physiological":    final_physio,
        }

        duration = time.time() - start
        result   = self.fusion.build_result(final_channels, raw_text=text, duration=duration)

        print(f"\n{'═'*55}")
        print(f"  TRUTH SCORE  : {result.truth_score:.1f}%")
        print(f"  VERDICT      : {result.verdict}")
        print(f"  RISK LEVEL   : {result.deception_risk:.1f}%")
        print(f"  FLAGS        : {len(result.flags)}")
        print(f"  DURATION     : {duration:.2f}s")
        print(f"{'═'*55}")

        return result

    def render_dashboard(self, result: AnalysisResult,
                          save_path: str = None) -> plt.Figure:
        """Render and optionally save the analysis dashboard."""
        fig = self.dashboard.render(result)
        if save_path:
            fig.savefig(save_path, dpi=150, bbox_inches="tight",
                        facecolor="#030812")
            print(f"  💾 Dashboard saved → {save_path}")
        plt.show()
        return fig


# Instantiate the full pipeline
pipeline = VeritasPipeline()

---
<a id="13-report"></a>
## 📄 Module 10 — Report Generator & Export

In [ ]:
# ============================================================
# REPORT GENERATOR
# ============================================================

class ReportGenerator:
    """
    Generates professional text and JSON reports from AnalysisResult.
    Supports:
    - Plain text report (terminal-friendly)
    - JSON export (machine-readable, full detail)
    - Pandas DataFrame summary (for further analysis)
    """

    @staticmethod
    def text_report(result: AnalysisResult) -> str:
        """Generate formatted text report."""
        sep = "═" * 62
        thin = "─" * 62

        lines = [
            "",
            sep,
            "  VERITAS — DECEPTION ANALYSIS REPORT",
            sep,
            f"  Session ID    : {result.session_id}",
            f"  Timestamp     : {result.timestamp.strftime('%Y-%m-%d %H:%M:%S')}",
            f"  Duration      : {result.duration_seconds:.2f} seconds",
            thin,
            f"  VERDICT       : {result.verdict}",
            f"  TRUTH SCORE   : {result.truth_score:.1f}%",
            f"  DECEPTION RISK: {result.deception_risk:.1f}%",
            thin,
            "  CHANNEL SCORES:",
        ]

        for name, cs in result.channels.items():
            bar_len = int(cs.truth_score / 5)
            bar     = "█" * bar_len + "░" * (20 - bar_len)
            lines.append(
                f"  {name:20s}  {bar}  {cs.truth_score:5.1f}%  [{cs.label}]"
            )

        lines += [
            thin,
            f"  DECEPTION FLAGS ({len(result.flags)}):",
        ]
        if result.flags:
            for flag in result.flags:
                lines.append(f"  ⚠  {flag}")
        else:
            lines.append("  ✅  No deception flags detected")

        if result.raw_text:
            lines += [
                thin,
                "  ANALYZED STATEMENT:",
                f"  "{result.raw_text[:200]}{'...' if len(result.raw_text) > 200 else ''}"",
            ]

        lines += [
            thin,
            "  DISCLAIMER: Probabilistic output only. Not legal evidence.",
            sep,
            "",
        ]
        return "\n".join(lines)

    @staticmethod
    def to_json(result: AnalysisResult, filepath: str = None) -> str:
        """Export full analysis as JSON."""
        data = {
            "session_id":      result.session_id,
            "timestamp":       result.timestamp.isoformat(),
            "duration_seconds":result.duration_seconds,
            "verdict":         result.verdict,
            "truth_score":     round(result.truth_score, 2),
            "deception_risk":  round(result.deception_risk, 2),
            "flags":           result.flags,
            "raw_text":        result.raw_text,
            "channels": {
                name: {
                    "truth_score":   round(cs.truth_score, 2),
                    "deception_risk":round(cs.deception_risk, 2),
                    "confidence":    round(cs.confidence, 3),
                    "label":         cs.label,
                    "flags":         cs.flags,
                    "metadata":      cs.metadata,
                }
                for name, cs in result.channels.items()
            },
            "timeline":        [round(v, 2) for v in result.timeline],
        }
        json_str = json.dumps(data, indent=2)
        if filepath:
            with open(filepath, "w") as f:
                f.write(json_str)
            print(f"  💾 JSON report saved → {filepath}")
        return json_str

    @staticmethod
    def to_dataframe(result: AnalysisResult) -> pd.DataFrame:
        """Convert channel scores to Pandas DataFrame."""
        rows = []
        for name, cs in result.channels.items():
            rows.append({
                "Channel":       name,
                "Truth Score":   round(cs.truth_score, 1),
                "Deception Risk":round(cs.deception_risk, 1),
                "Confidence":    round(cs.confidence, 3),
                "Verdict":       cs.label,
                "Flags":         len(cs.flags),
            })
        df = pd.DataFrame(rows).set_index("Channel")
        return df

    @staticmethod
    def save_all(result: AnalysisResult, base_path: str = "veritas_report"):
        """Save text report, JSON, and CSV."""
        # Text
        with open(f"{base_path}.txt", "w") as f:
            f.write(ReportGenerator.text_report(result))
        print(f"  💾 Text report   → {base_path}.txt")
        # JSON
        ReportGenerator.to_json(result, f"{base_path}.json")
        # CSV
        df = ReportGenerator.to_dataframe(result)
        df.to_csv(f"{base_path}.csv")
        print(f"  💾 CSV summary   → {base_path}.csv")
        print("  ✅ All reports exported")


reporter = ReportGenerator()
print("✅ ReportGenerator ready")
print("   Outputs: .txt report | .json export | .csv dataframe")

---
<a id="14-demo"></a>
## 🧪 Section 14 — DEMO: Full Synthetic Simulation

Run three complete analyses at different stress levels and compare results.

In [ ]:
# ============================================================
# DEMO — CASE 1: HIGH DECEPTION (Stress Level: 85%)
# ============================================================

print("\n" + "🔴 " * 20)
print("CASE 1 — HIGH DECEPTION SUBJECT")
print("🔴 " * 20)

result_deceptive = pipeline.analyze_synthetic(
    n_frames         = 100,
    n_audio_segments = 50,
    text = (
        "Trust me, I swear I wasn't anywhere near that location. "
        "You have to believe me, I would never do something like that, honestly. "
        "Like, I don't know, maybe someone else was there, I'm not sure, "
        "sort of hard to remember exactly but I definitely wasn't involved."
    ),
    stress_level     = 0.85,
    response_latency = 1.4,
)

print(ReportGenerator.text_report(result_deceptive))
df1 = ReportGenerator.to_dataframe(result_deceptive)
display(df1.style.background_gradient(cmap="RdYlGn", subset=["Truth Score"],
                                       vmin=0, vmax=100)
              .format({"Truth Score": "{:.1f}%",
                       "Deception Risk": "{:.1f}%",
                       "Confidence": "{:.0%}"})
              .set_caption("Case 1 — High Deception"))

In [ ]:
# ============================================================
# DEMO — CASE 2: TRUTHFUL SUBJECT (Stress Level: 15%)
# ============================================================

print("\n" + "✅ " * 20)
print("CASE 2 — TRUTHFUL SUBJECT")
print("✅ " * 20)

result_truthful = pipeline.analyze_synthetic(
    n_frames         = 100,
    n_audio_segments = 50,
    text = (
        "I was at the downtown office at 3pm on Tuesday the 14th of March. "
        "I had a 90-minute budget meeting with Sarah Chen and James Patel "
        "from accounting. We reviewed the Q1 figures — revenue was up 12%. "
        "I left at 4:35pm, drove home on Route 7, and arrived at 5:20pm."
    ),
    stress_level     = 0.15,
    response_latency = 0.35,
)

print(ReportGenerator.text_report(result_truthful))
df2 = ReportGenerator.to_dataframe(result_truthful)
display(df2.style.background_gradient(cmap="RdYlGn", subset=["Truth Score"],
                                       vmin=0, vmax=100)
              .format({"Truth Score": "{:.1f}%",
                       "Deception Risk": "{:.1f}%",
                       "Confidence": "{:.0%}"})
              .set_caption("Case 2 — Truthful Subject"))

In [ ]:
# ============================================================
# DEMO — CASE 3: INCONCLUSIVE / MIXED SIGNALS (Stress Level: 45%)
# ============================================================

print("\n" + "🟡 " * 20)
print("CASE 3 — INCONCLUSIVE / MIXED SIGNALS")
print("🟡 " * 20)

result_mixed = pipeline.analyze_synthetic(
    n_frames         = 100,
    n_audio_segments = 50,
    text = (
        "I think I was probably at home that evening, or maybe out. "
        "I'm not entirely sure about the exact time, but I believe "
        "it was sometime around 7 or 8pm. I usually stay in on weekdays."
    ),
    stress_level     = 0.45,
    response_latency = 0.75,
)

print(ReportGenerator.text_report(result_mixed))
df3 = ReportGenerator.to_dataframe(result_mixed)
display(df3.style.background_gradient(cmap="RdYlGn", subset=["Truth Score"],
                                       vmin=0, vmax=100)
              .format({"Truth Score": "{:.1f}%",
                       "Deception Risk": "{:.1f}%",
                       "Confidence": "{:.0%}"})
              .set_caption("Case 3 — Inconclusive / Mixed"))

In [ ]:
# ============================================================
# COMPARATIVE DASHBOARD — All 3 Cases
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(22, 7), facecolor="#030812")
fig.suptitle("VERITAS — COMPARATIVE ANALYSIS: 3 SUBJECTS",
             fontsize=14, fontweight="bold", color=CYAN,
             fontfamily="monospace", y=1.01)

cases = [
    ("SUBJECT A — DECEPTIVE",  result_deceptive, RED),
    ("SUBJECT B — TRUTHFUL",   result_truthful,  GREEN),
    ("SUBJECT C — INCONCLUSIVE", result_mixed,   YELLOW),
]

for ax, (title, result, c) in zip(axes, cases):
    ax.set_facecolor("#080f1e")
    ax.set_title(title, color=c, fontsize=9, fontfamily="monospace", pad=10)

    names  = list(result.channels.keys())
    scores = [result.channels[n].truth_score for n in names]
    colors = [color_from_score(s) for s in scores]

    y_pos = range(len(names))
    ax.barh(y_pos, scores, color=colors, alpha=0.75, height=0.6, edgecolor="none")
    ax.set_yticks(list(y_pos))
    ax.set_yticklabels([n.split()[0] for n in names], fontsize=8, fontfamily="monospace")
    ax.set_xlim(0, 105)
    ax.set_xlabel("Truth Score %", color=DIM, fontsize=8)
    ax.tick_params(colors=DIM)
    ax.axvline(50, color=YELLOW, linewidth=0.7, linestyle="--", alpha=0.5)
    ax.axvline(75, color=GREEN,  linewidth=0.7, linestyle="--", alpha=0.5)
    ax.grid(axis="x", alpha=0.15)

    for i, s in enumerate(scores):
        ax.text(s + 1, i, f"{s:.0f}%", va="center", fontsize=8,
                color=colors[i], fontfamily="monospace", fontweight="bold")

    # Overall score badge
    ax.text(0.5, -0.18,
            f"OVERALL: {result.truth_score:.1f}%  —  {result.verdict}",
            transform=ax.transAxes, ha="center", fontsize=8,
            color=c, fontfamily="monospace", fontweight="bold",
            bbox=dict(boxstyle="round", facecolor="#080f1e", edgecolor=c, alpha=0.8))

plt.tight_layout()
plt.savefig("veritas_comparison.png", dpi=150, bbox_inches="tight",
            facecolor="#030812")
plt.show()
print("💾 Comparison chart saved → veritas_comparison.png")

In [ ]:
# ============================================================
# RENDER FULL DASHBOARD — Most Interesting Case
# ============================================================

print("\n📊 Rendering full VERITAS dashboard for deceptive subject...")
fig = pipeline.render_dashboard(result_deceptive, save_path="veritas_dashboard.png")

In [ ]:
# ============================================================
# EXPORT ALL REPORTS
# ============================================================

print("📄 Exporting reports for all 3 cases...\n")

ReportGenerator.save_all(result_deceptive, "report_deceptive")
print()
ReportGenerator.save_all(result_truthful,  "report_truthful")
print()
ReportGenerator.save_all(result_mixed,     "report_mixed")

print("\n✅ All reports exported successfully!")
print("   Files: report_*.txt | report_*.json | report_*.csv")
print("   Images: veritas_dashboard.png | veritas_comparison.png")

In [ ]:
# ============================================================
# QUICK TEXT ANALYSIS — Analyze any statement
# ============================================================

def quick_analyze(statement: str, latency: float = 0.5):
    """
    Quick NLP analysis of any text statement.
    Usage: quick_analyze("your statement here")
    """
    result = pipeline.analyze_text(statement, response_latency=latency)
    print(ReportGenerator.text_report(result))
    return result

# ── Try your own statement ─────────────────────────────────────
my_statement = "I was home all night. I didn't go anywhere."

result = quick_analyze(my_statement, latency=0.4)

---

<div align="center">

```
VERITAS v2.4  ·  Python Notebook Edition
Micro-Expressions · Eye Movement · Voice Stress · NLP · Delay · Physiological
Zero external API calls · All processing local
```

</div>